In [8]:
import csv
from datetime import datetime
import requests
from bs4 import BeautifulSoup, SoupStrainer
import httplib2
import re
import pandas as pd

In [9]:
baselink = "https://ca.indeed.com"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36"}

In [10]:
df = pd.DataFrame(columns=['Title', 'Location', 'Extract_date', 'Link', 'Desc', 
                           'Company', 'CompanyLink', 'Salary', 'Job_Type1',
                           'Job_Type2', 'Remote', 'Reviews', 'Stars'])

In [11]:
# Generate url from position, location and date posted.
def get_url(position, location, dateposted):
    template = 'https://ca.indeed.com/jobs?q={}&l={}&fromage={}'
    position = position.replace(' ', '+')
    location = location.replace(' ', '+')
    dateposted = dateposted.replace(' ', '+')
    url = template.format(position, location, dateposted)
    return url
url = get_url('data analyst', 'canada', '1')
print(url)

https://ca.indeed.com/jobs?q=data+analyst&l=canada&fromage=1


In [12]:
for number in range(0, 10, 10):
    links = url+'&start={}'.format(number)
    r = requests.get(links, headers = headers)
    soup = BeautifulSoup(r.content, 'html.parser')
    jobcards = soup.find("div", attrs={"id": "mosaic-zone-jobcards"})
    links = []
    
    for l in jobcards.find_all('a', attrs={'href': re.compile("js")}):
        links.append(l.get('href'))
        
    for n in links:
        newlink = baselink + str(n)
        r = requests.get(newlink, headers = headers)
        soup = BeautifulSoup(r.text, 'html.parser')
        extract_date = datetime.today().strftime('%Y-%m-%d')
        Loc = soup.find(class_="icl-u-xs-mt--xs icl-u-textColor--secondary jobsearch-JobInfoHeader-subtitle jobsearch-DesktopStickyContainer-subtitle")
        comp = soup.find(class_="jobsearch-CompanyInfoContainer")
        sal = soup.find(class_="jobsearch-JobMetadataHeader-item")
        rat = soup.find(class_="icl-u-lg-block icl-u-lg-mr--sm icl-u-xs-hide")
        
        try:
            reviews = rat.find('div', string=re.compile("reviews")).text.strip()
        except AttributeError:
            reviews = "NA"
                        
        try:
            stars = rat.select_one('meta[content]')['content']
        except AttributeError:
            stars = "NA"
                        
        try:
            desc = soup.find(class_="jobsearch-jobDescriptionText").text.strip()
        except AttributeError:
            desc = "NA"
                        
        try:
            job_title = soup.find(class_="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title").text.strip()
        except AttributeError:
            job_title = "NA"
                        
        try:
            job_type1 = sal.find('span', string=re.compile("Full|Part|Casu|Intern")).text.strip()
        except AttributeError:
            job_type1 = "NA"
            
        try:
            job_type2 = sal.find('span', string=re.compile("Perman|Contr|term|Temp|Freel|Apprent")).text.strip()
        except AttributeError:
            job_type2 = "NA"
                        
        try:
            remote = Loc.find('div', string=re.compile("Remote|remote")).text.strip()
        except AttributeError:
            remote = "NA"
                        
        try:
            salary = sal.find('span', string=re.compile("year|hour")).text.strip()
        except AttributeError:
            salary = "NA"
                        
        try:
            companylink = comp.find('a', attrs={'href': re.compile("//")}).get('href')
        except AttributeError:
            companylink = "NA"
                        
        try:
            company = comp.find('a', attrs={'href': re.compile("//")}).string.strip()
        except AttributeError:
            company = "NA"
                        
        try:
            location = Loc.find('div', string=re.compile("ON|SK|BC|AB|PE|NS|MB|QC")).string.strip()
        except AttributeError:
            location = "NA"
                        
        df.loc[len(df.index)] = [job_title, location, extract_date,  newlink, desc, company, companylink, salary, job_type1, job_type2, remote, reviews, stars]

In [13]:
df.to_csv(r'indeedjobs.csv', index = False)